In [ ]:
import numpy as np
import numba as nb

import ice_funcs as icef

## Todo:

- **FIGURE OUT PROPER SYNTAX FOR DIFFUSION RULES (width and length restricted)**
- Appliquer la diffusion
- Gérer les conditions de diffusion
- Workout comment intégrer le filling field 'f'
- Continue commenting in `ice_funcs.py`

In [ ]:
# CONSTANTS

L = 1001
W = (L+1)//2
INITIAL_SAT = 1

In [ ]:
test_neighbor_array = icef.construct_neighbor_array(L, W)
test_ice_map = icef.construct_minimal_ice_map(L, W)
test_boundary_map = icef.construct_boundary_map(test_ice_map, test_neighbor_array)
test_diffusion_rules = icef.construct_default_diffusion_rules(L,W)
test_initial_sat_map = icef.initialize_sat_map(L,W)


print(test_boundary_map[-5:,:5])
# %timeit icef.relax_sat_map(test_initial_sat_map, test_diffusion_rules, test_ice_map, test_boundary_map, test_neighbor_array, L)


In [ ]:
# print(test_neighbor_array)
# print(test_ice_map)


In [ ]:
@nb.njit
def change_element():
    arr = np.array([0,0])
    arr[1] = 1

@nb.njit
def append_element():
    arr = np.array([0], dtype=np.int8)
    arr = np.append(arr, 1)

In [ ]:
import matplotlib.pyplot as plt
import matplotlib as mpl
import numpy as np

mpl.rcParams["figure.dpi"] = 300

plt.plot([0,0,0, np.sqrt(3)/(2*20)],[0,0.05,0.1, 0.025], marker="H", linewidth=0)

plt.xlim(-1,1)
plt.ylim(-1,1)

plt.gca().set_aspect(1)